In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [14]:
sample = " if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to log for the endless immensity of the sea"

idx2char = list(set(sample))
char2idx = {c: i for i, c in enumerate(idx2char)}

dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1


sample_idx = [char2idx[c] for c in sample]

x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

x_one_hot = tf.one_hot(X, num_classes)

In [16]:
with tf.variable_scope("cell2", reuse=None) as scope:
    cell = tf.contrib.rnn.BasicLSTMCell(
        num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)


    X_for_fc = tf.reshape(outputs, [-1, hidden_size])
    outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

    # reshape out for sequence_loss
    outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])


weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in tqdm(range(50), leave=True):
        l, _ = sess.run([loss, optimizer], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        
        if i % 1 == 0:
            print(i, "loss:", l, "\tPrediction:", ''.join(result_str))

0 loss: 2.30972 	Prediction:                
1 loss: 2.14673 	Prediction:                
2 loss: 2.04982 	Prediction:  o  ou       oo
3 loss: 1.89593 	Prediction:  o  ou       ou
4 loss: 1.71309 	Prediction:  o  ou       ou
5 loss: 1.48897 	Prediction: yo you w    you
6 loss: 1.24397 	Prediction: yo you w n  you
7 loss: 0.99313 	Prediction: yo you want you
8 loss: 0.772934 	Prediction: yf you want you
9 loss: 0.578643 	Prediction: yf you want you
10 loss: 0.425183 	Prediction: if you want you
11 loss: 0.312506 	Prediction: if you want you
12 loss: 0.232103 	Prediction: if you want you
13 loss: 0.175003 	Prediction: if you want you
14 loss: 0.130937 	Prediction: if you want you
15 loss: 0.0962018 	Prediction: if you want you
16 loss: 0.0695674 	Prediction: if you want you
17 loss: 0.0498363 	Prediction: if you want you
18 loss: 0.0355209 	Prediction: if you want you
19 loss: 0.0252949 	Prediction: if you want you
20 loss: 0.0181505 	Prediction: if you want you
21 loss: 0.0132617 	Predi